In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
#Count the number of images in the respective classes - 0 - Brain Tumor and 1 - Healthy
ROOT_DIR = "/content/drive/MyDrive/Dataset/Brain_Tumor_Detection"
number_of_images = {}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))
number_of_images.items()


In [4]:
len(os.listdir("/content/drive/MyDrive/Dataset/Brain_Tumor_Detection"))

2

In [8]:
def dataFolder(p, split):
  #Creating train Folder
  if not os.path.exists('./'+p):
    os.mkdir('./' + p)

    for dir in os.listdir(ROOT_DIR):
      os.makedirs('./' + p + "/" +dir)

      for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)),
                                size = (math.floor(split * number_of_images[dir])-5),
                                replace = False):
        O = os.path.join(ROOT_DIR, dir, img)
        D = os.path.join("./"+p , dir)
        shutil.copy(O, D)
        os.remove(O)
  else:
    print(f"{p} folder exists")


In [6]:
dataFolder('train', 0.7)

In [7]:
dataFolder('val', 0.15)

In [8]:
dataFolder('test', 0.15)

In [ ]:
import os

folder_path = '/content/train/yes'  # Path to the folder containing images
image_files = os.listdir(folder_path)

# Print the first 5 image filenames to confirm
print(image_files[:5])


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = os.path.join(folder_path, image_files[0])

img = image.load_img(img_path)

img_array = np.array(img)

print(img_array.shape)


Preparing Data


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.applications.mobilenet import preprocess_input

In [18]:
def preprocessingImages(path):
  """
  input : path
  output: preprocessed images
  """
  image_data = ImageDataGenerator(zoom_range = 0.2, shear_range = 0.2, preprocessing_function = preprocess_input, horizontal_flip = True)
  image = image_data.flow_from_directory(directory = path, target_size = (224, 224), batch_size = 32, class_mode = 'binary')
  return image

In [19]:
def preprocessingImages2(path):
  """
  input : path
  output: preprocessed images
  """
  image_data = ImageDataGenerator(preprocessing_function = preprocess_input)
  image = image_data.flow_from_directory(directory = path, target_size = (224, 224), batch_size = 32, class_mode = 'binary')
  return image

In [ ]:
path = '/content/train'
train_data = preprocessingImages(path)

In [ ]:

path = '/content/test'
test_data = preprocessingImages2(path)

In [ ]:
path = '/content/val'
val_data = preprocessingImages2(path)

In [ ]:
train_data.class_indices

Model Training with Transfer Learning

In [32]:
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
from keras.layers import Flatten, Dense
from keras.models import Model, load_model
from keras.applications.mobilenet import MobileNet, preprocess_input


In [ ]:
base_model = MobileNet(input_shape = (224, 224,3), include_top = False)

In [25]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
base_model.summary()

In [29]:
X = base_model.output
X = Flatten()(X)
X = Dense(units = 1, activation ='sigmoid')(X)

model = Model(base_model.input, X)

In [ ]:
model.summary()

In [33]:
model.compile(optimizer = 'rmsprop', loss = keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [35]:
#Call back

from keras.callbacks import ModelCheckpoint, EarlyStopping

mc = ModelCheckpoint(monitor = 'val_accuracy', filepath = '/content/best_model.keras', verbose = 1, save_best_only = 1)

es = EarlyStopping(monitor = 'val_accuracy', min_delta = 0.01, patience = 3, verbose = 1)

cb = [mc, es]

In [ ]:
hist = model.fit(train_data,
                 steps_per_epoch = 8,
                 epochs = 20,
                 validation_data = val_data,
                 validation_steps = 16,
                 callbacks = cb)

In [39]:
model.save('/content/best_model.keras')

In [40]:
#load the best model

model = load_model('./best_model.keras')

In [ ]:
#Accuracy

acc = model.evaluate(test_data)

print(f"Our model accuracy is: {acc * 100}%")

In [ ]:
h = hist.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c = 'red')
plt.title('acc vs val_accuracy')
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c = 'red')
plt.title('loss vs val_loss')
plt.show()